Following: 

nilssonTurbulenceKineticEnergy2016

https://acp.copernicus.org/articles/16/8849/2016/

In [141]:
import numpy as np
import xarray as xr

import datetime as dt
import os
import pandas as pd

import matplotlib.pyplot as plt

import altair as alt
alt.data_transformers.enable('json')

from sublimpy import utils
import glob
import pytz
import re
from scipy.signal import welch, csd
from scipy.stats import chi2
from process_fast_data.fast_data_calculate_spectra_nomrd import calculate_mrd_for_df, fast_data_files_to_dataframe
from sublimpy import extrautils

In [142]:
# DATA_DIR = "/storage/elilouis/sublimationofsnow/"
DATA_DIR = "/Users/elischwat/Development/data/sublimationofsnow/"

# DATES = pd.Series(['20230418', '20230419'])
# DATE_LOCAL = '20230418'

# DATES = pd.Series(['20230409', '20230410'])
# DATE_LOCAL = '20230409'

# Open SOS Fast EC data

Gather files

In [143]:
## USE THE ORIGINAL FILES
###########################
sos_file_list = sorted(np.array([
    [f for f in glob.glob(
        os.path.join(DATA_DIR, "sosqc_fast/isfs_sos_qc_geo_tiltcor_hr_v2_**.nc")
    ) if d in f]
    for d in DATES
]).flatten())

# Just grab 24 hours from a single day IN LOCAL TIME
sos_file_list = sos_file_list[6:30]

Iterate over files, opening them and applying double rotation for SEPARATE HOURS

In [144]:
df_list = []
for file in sos_file_list:
    df_list.append(
        fast_data_files_to_dataframe(
            [file],  
            rotation='double'
        )
    )
fast_df_sos_all_data = pd.concat(df_list)

-10.44586
mean u, v, w after first rotation  u_2m_c    2.534782e+00
v_2m_c    1.695421e-08
w_2m_c    2.252408e-02
dtype: float32
-9.058819
mean u, v, w after first rotation  u_3m_c    2.731780e+00
v_3m_c    2.882216e-08
w_3m_c    2.350053e-02
dtype: float32
-11.7984295
mean u, v, w after first rotation  u_5m_c    2.973845e+00
v_5m_c   -5.594889e-08
w_5m_c    2.612678e-02
dtype: float32
-10.98607
mean u, v, w after first rotation  u_10m_c    3.130796e+00
v_10m_c    2.373589e-08
w_10m_c   -8.393890e-02
dtype: float32
-13.900157
mean u, v, w after first rotation  u_15m_c    3.405484e+00
v_15m_c    5.764431e-08
w_15m_c    6.672936e-02
dtype: float32
-15.9647875
mean u, v, w after first rotation  u_20m_c    3.288025e+00
v_20m_c   -6.103516e-08
w_20m_c   -5.654476e-02
dtype: float32
-11.012845
mean u, v, w after first rotation  u_3m_uw    2.399846e+00
v_3m_uw    2.627903e-08
w_3m_uw    6.407025e-03
dtype: float32
-11.17456
mean u, v, w after first rotation  u_10m_uw    3.118312e+00
v_10m_uw 

In [145]:
fast_df_sos_all_data = utils.modify_df_timezone(fast_df_sos_all_data, 'UTC', 'US/Mountain')
fast_df_sos_all_data = fast_df_sos_all_data.set_index('time').loc[DATE_LOCAL]

# Apply planar fits (pre-calculated ones, we open the parameters up and apply them)

In [62]:
# fits = pd.read_csv(
#     "/Users/elischwat/Development/data/sublimationofsnow/monthly_planar_fits.csv",
#     sep='\s+'
# )
# fits = fits.set_index(['month', 'height', 'tower'])
# fits = fits.loc[fast_df_sos_all_data.index.month[0], :, 'c']

In [63]:
# for H in [2,3,5,10,15,20]:
#     (
#         u_fit , v_fit , w_fit
#     ) = extrautils.apply_planar_fit(
#         fast_df_sos_all_data[f'u_{H}m_c'], 
#         fast_df_sos_all_data[f'v_{H}m_c'], 
#         fast_df_sos_all_data[f'w_{H}m_c'], 
#         fits.loc[f'{H}']['a'], 
#         fits.loc[f'{H}'][['W_f_1', 'W_f_2', 'W_f_3']].values
#     )
#     fast_df_sos_all_data[f'u_{H}m_c'] = u_fit
#     fast_df_sos_all_data[f'v_{H}m_c'] = v_fit
#     fast_df_sos_all_data[f'w_{H}m_c'] = w_fit

# Remove outliers

In [146]:
# Remove outliers, defined as outside ±4 standard deviations from the mean value for each hour
def remove_outliers(series):
    mean = series.mean()
    stddev = series.std()
    return series.where(
        (series < mean + 4*stddev)
        &
        (series > mean - 4*stddev)
    )

for col in fast_df_sos_all_data.columns:
    fast_df_sos_all_data[col] = fast_df_sos_all_data[col].groupby(
        pd.Grouper(freq='1h')
    ).apply(remove_outliers).values

# Calculate TKE variables

In [147]:
fast_df_sos_all_data['tke_2m_c'] = (
    0.5*(fast_df_sos_all_data[['u_2m_c', 'v_2m_c', 'w_2m_c']]**2).sum(axis=1)
)
fast_df_sos_all_data['tke_3m_c'] = (
    0.5*(fast_df_sos_all_data[['u_3m_c', 'v_3m_c', 'w_3m_c']]**2).sum(axis=1)
)
fast_df_sos_all_data['tke_5m_c'] = (
    0.5*(fast_df_sos_all_data[['u_5m_c', 'v_5m_c', 'w_5m_c']]**2).sum(axis=1)
)
fast_df_sos_all_data['tke_10m_c'] = (
    0.5*(fast_df_sos_all_data[['u_10m_c', 'v_10m_c', 'w_10m_c']]**2).sum(axis=1)
)
fast_df_sos_all_data['tke_15m_c'] = (
    0.5*(fast_df_sos_all_data[['u_15m_c', 'v_15m_c', 'w_15m_c']]**2).sum(axis=1)
)
fast_df_sos_all_data['tke_20m_c'] = (
    0.5*(fast_df_sos_all_data[['u_20m_c', 'v_20m_c', 'w_20m_c']]**2).sum(axis=1)
)

# Calculate Reynolds Averages

In [148]:
SAMPLES_PER_AVERAGING_LENGTH = 20*60*10
def create_re_avg_ds(ds, var1,  var2, covariance_name):
    # Function to do Reynolds Averaging
    coarse_ds = ds.coarsen(time=SAMPLES_PER_AVERAGING_LENGTH).mean(skipna=True)
    coarse_ds = coarse_ds.assign_coords(time = coarse_ds.time.dt.round('1s'))
    coarse_ds = coarse_ds.reindex_like(ds, method='nearest')
    ds[f"{var1}_mean"] = coarse_ds[f"{var1}"]
    ds[f"{var1}_fluc"] = ds[f"{var1}"] - ds[f"{var1}_mean"]
    ds[f"{var2}_mean"] = coarse_ds[f"{var2}"]
    ds[f"{var2}_fluc"] = ds[f"{var2}"] - ds[f"{var2}_mean"]
    ds[covariance_name] = ds[f"{var2}_fluc"] * ds[f"{var1}_fluc"]
    ds = ds.coarsen(time = SAMPLES_PER_AVERAGING_LENGTH).mean()
    ds = ds.assign_coords(time = ds.time.dt.round('1s'))
    return ds.to_dataframe()

In [149]:
budget_df = pd.DataFrame()
for H in [2, 3, 5, 10, 15, 20]:
    momentum_flux_streamwise_df = create_re_avg_ds(
        fast_df_sos_all_data.to_xarray(), 
        f'u_{H}m_c', f'w_{H}m_c', f'u_w__{H}m_c'
    )[[f'u_w__{H}m_c', f'u_{H}m_c_mean']]
    momentum_flux_crosswise_df = create_re_avg_ds(
        fast_df_sos_all_data.to_xarray(), 
        f'v_{H}m_c', f'w_{H}m_c', f'v_w__{H}m_c'
    )[[f'v_w__{H}m_c', f'v_{H}m_c_mean']]
    tke_flux_vertical_df = create_re_avg_ds(
        fast_df_sos_all_data.to_xarray(), 
        f'w_{H}m_c', f'tke_{H}m_c', f'w_tke__{H}m_c'
    )[[f'w_tke__{H}m_c', f'tke_{H}m_c']]
    sensibleheat_flux_vertical_df = create_re_avg_ds(
        fast_df_sos_all_data.to_xarray(), 
        f'w_{H}m_c', f'tc_{H}m_c', f'w_tc__{H}m_c'
    )[[f'w_tc__{H}m_c', f'tc_{H}m_c']]
    new_df = momentum_flux_streamwise_df.join(momentum_flux_crosswise_df).join(tke_flux_vertical_df).join(sensibleheat_flux_vertical_df)
    # Handle the initial lack of indices to join on
    if len(budget_df) == 0:
        budget_df = new_df
    else:
        budget_df = budget_df.join(new_df)

# Calculate wind component gradients
Shear Production (S)
$$S = -\overline{u'w'} \frac{d \overline{u}}{d z} -\overline{v'w'} \frac{d \overline{v}}{d z}$$

In [150]:
from sublimpy.gradients import LogPolynomial

In [151]:
new_cols_df = pd.DataFrame()
for index_ts, values in budget_df.iterrows():
    # calculate gradients for u
    (a,b,c) = LogPolynomial.fit_function(
        list(values[['u_2m_c_mean', 'u_3m_c_mean', 'u_5m_c_mean', 'u_10m_c_mean', 'u_15m_c_mean', 'u_20m_c_mean']].values),
        [2, 3, 5, 10, 15, 20]
    )
    du_dz_2m_c = LogPolynomial.gradient_single_component(2, a, b)
    du_dz_3m_c = LogPolynomial.gradient_single_component(3, a, b)
    du_dz_5m_c = LogPolynomial.gradient_single_component(5, a, b)
    du_dz_10m_c = LogPolynomial.gradient_single_component(10, a, b)
    du_dz_15m_c = LogPolynomial.gradient_single_component(15, a, b)
    du_dz_20m_c = LogPolynomial.gradient_single_component(20, a, b)

    # calculate gradients for u
    (a,b,c) = LogPolynomial.fit_function(
        list(values[['v_2m_c_mean', 'v_3m_c_mean', 'v_5m_c_mean', 'v_10m_c_mean', 'v_15m_c_mean', 'v_20m_c_mean']].values),
        [2, 3, 5, 10, 15, 20]
    )
    dv_dz_2m_c = LogPolynomial.gradient_single_component(2, a, b)
    dv_dz_3m_c = LogPolynomial.gradient_single_component(3, a, b)
    dv_dz_5m_c = LogPolynomial.gradient_single_component(5, a, b)
    dv_dz_10m_c = LogPolynomial.gradient_single_component(10, a, b)
    dv_dz_15m_c = LogPolynomial.gradient_single_component(15, a, b)
    dv_dz_20m_c = LogPolynomial.gradient_single_component(20, a, b)

    new_cols_df = pd.concat([ 
        new_cols_df,
        pd.DataFrame({
            'time':         [index_ts],
            'du_dz_2m_c':   [du_dz_2m_c],
            'du_dz_3m_c':   [du_dz_3m_c],
            'du_dz_5m_c':   [du_dz_5m_c],
            'du_dz_10m_c':  [du_dz_10m_c],
            'du_dz_15m_c':  [du_dz_15m_c],
            'du_dz_20m_c':  [du_dz_20m_c],
            'dv_dz_2m_c':   [dv_dz_2m_c],
            'dv_dz_3m_c':   [dv_dz_3m_c],
            'dv_dz_5m_c':   [dv_dz_5m_c],
            'dv_dz_10m_c':  [dv_dz_10m_c],
            'dv_dz_15m_c':  [dv_dz_15m_c],
            'dv_dz_20m_c':  [dv_dz_20m_c],
        })
    ])

In [152]:
budget_df = budget_df.join(new_cols_df.set_index('time'))

# Calculate tke flux gradients
Transport (T)
$$T = - \frac{d}{d z}  \overline{w'e'}$$

In [153]:
new_cols_df = pd.DataFrame()
for index_ts, values in budget_df.iterrows():
    # calculate gradients for u
    (a,b,c) = LogPolynomial.fit_function(
        list(values[['w_tke__2m_c', 'w_tke__3m_c', 'w_tke__5m_c', 'w_tke__10m_c', 'w_tke__15m_c', 'w_tke__20m_c']].values),
        [2, 3, 5, 10, 15, 20]
    )

    dtkeflux_dz_2m_c = LogPolynomial.gradient_single_component(2, a, b)
    dtkeflux_dz_3m_c = LogPolynomial.gradient_single_component(3, a, b)
    dtkeflux_dz_5m_c = LogPolynomial.gradient_single_component(5, a, b)
    dtkeflux_dz_10m_c = LogPolynomial.gradient_single_component(10, a, b)
    dtkeflux_dz_15m_c = LogPolynomial.gradient_single_component(15, a, b)
    dtkeflux_dz_20m_c = LogPolynomial.gradient_single_component(20, a, b)

    new_cols_df = pd.concat([ 
        new_cols_df,
        pd.DataFrame({
            'time':         [index_ts],
            'dtkeflux_dz_2m_c':   [dtkeflux_dz_2m_c],
            'dtkeflux_dz_3m_c':   [dtkeflux_dz_3m_c],
            'dtkeflux_dz_5m_c':   [dtkeflux_dz_5m_c],
            'dtkeflux_dz_10m_c':  [dtkeflux_dz_10m_c],
            'dtkeflux_dz_15m_c':  [dtkeflux_dz_15m_c],
            'dtkeflux_dz_20m_c':  [dtkeflux_dz_20m_c],
        })
    ])

In [154]:
budget_df = budget_df.join(new_cols_df.set_index('time'))

# Calculate shear velocity

In [155]:
budget_df['u*_2m_c'] = (budget_df['u_w__2m_c']**2 + budget_df['v_w__2m_c']**2)**0.25
budget_df['u*_3m_c'] = (budget_df['u_w__3m_c']**2 + budget_df['v_w__3m_c']**2)**0.25
budget_df['u*_5m_c'] = (budget_df['u_w__5m_c']**2 + budget_df['v_w__5m_c']**2)**0.25
budget_df['u*_10m_c'] = (budget_df['u_w__10m_c']**2 + budget_df['v_w__10m_c']**2)**0.25
budget_df['u*_15m_c'] = (budget_df['u_w__15m_c']**2 + budget_df['v_w__15m_c']**2)**0.25
budget_df['u*_20m_c'] = (budget_df['u_w__20m_c']**2 + budget_df['v_w__20m_c']**2)**0.25

# Calculate terms

$$T = - \frac{d}{d z}  \overline{w'e'}$$
$$S = -\overline{u'w'} \frac{d \overline{u}}{d z} -\overline{v'w'} \frac{d \overline{v}}{d z}$$
$$B = \frac{g}{\overline{\theta}} \overline{w'\theta_v'}$$

In [156]:
# open up the 30min acverages so we can use potential temp as a reference temperature
sos_df = pd.read_parquet("../paper1/process_slow_data/tidy_df_20221101_20230619_planar_fit_multiplane_q7_flags9000_pf10.parquet")
sos_df_localtime = utils.modify_df_timezone(
    sos_df,
    'UTC',
    'US/Mountain'
)
# grab the measurement we want to use 
tpot_timeseries = sos_df_localtime.query("variable == 'Tpot_10m_c'").set_index('time')[['value']]
# resample to match our 10min data
tpot_timeseries = tpot_timeseries.drop_duplicates()
tpot_timeseries = tpot_timeseries.resample('10min').mean()
tpot_timeseries.value = tpot_timeseries.value.ffill()
tpot_timeseries.index = tpot_timeseries.index + dt.timedelta(minutes=5)

budget_df = budget_df.join(
    tpot_timeseries.value.rename('Tpot_10m_c'),
    how='left'
)

budget_df['Tpot_10m_c'] = budget_df['Tpot_10m_c'] + 273.15

In [157]:

reference_temp = budget_df['Tpot_10m_c']
g = 9.81
B_terms_df = pd.concat([
    pd.DataFrame((( g / reference_temp ) * budget_df['w_tc__2m_c']).rename('B')).assign(height = 2),
    pd.DataFrame((( g / reference_temp ) * budget_df['w_tc__3m_c']).rename('B')).assign(height = 3),
    pd.DataFrame((( g / reference_temp ) * budget_df['w_tc__5m_c']).rename('B')).assign(height = 5),
    pd.DataFrame((( g / reference_temp ) * budget_df['w_tc__10m_c']).rename('B')).assign(height = 10),
    pd.DataFrame((( g / reference_temp ) * budget_df['w_tc__15m_c']).rename('B')).assign(height = 15),
    pd.DataFrame((( g / reference_temp ) * budget_df['w_tc__20m_c']).rename('B')).assign(height = 20),
])

T_terms_df = pd.concat([
    pd.DataFrame((- budget_df['dtkeflux_dz_2m_c']).rename('T')).assign(height = 2),
    pd.DataFrame((- budget_df['dtkeflux_dz_3m_c']).rename('T')).assign(height = 3),
    pd.DataFrame((- budget_df['dtkeflux_dz_5m_c']).rename('T')).assign(height = 5),
    pd.DataFrame((- budget_df['dtkeflux_dz_10m_c']).rename('T')).assign(height = 10),
    pd.DataFrame((- budget_df['dtkeflux_dz_15m_c']).rename('T')).assign(height = 15),
    pd.DataFrame((- budget_df['dtkeflux_dz_20m_c']).rename('T')).assign(height = 20)
])
S_terms_df = pd.concat([
    pd.DataFrame((- budget_df['u_w__2m_c'] * budget_df['du_dz_2m_c'] - budget_df['v_w__2m_c'] * budget_df['dv_dz_2m_c']).rename('S')).assign(height=2),
    pd.DataFrame((- budget_df['u_w__3m_c'] * budget_df['du_dz_3m_c'] - budget_df['v_w__3m_c'] * budget_df['dv_dz_3m_c']).rename('S')).assign(height=3),
    pd.DataFrame((- budget_df['u_w__5m_c'] * budget_df['du_dz_5m_c'] - budget_df['v_w__5m_c'] * budget_df['dv_dz_5m_c']).rename('S')).assign(height=5),
    pd.DataFrame((- budget_df['u_w__10m_c'] * budget_df['du_dz_10m_c'] - budget_df['v_w__10m_c'] * budget_df['dv_dz_10m_c']).rename('S')).assign(height=10),
    pd.DataFrame((- budget_df['u_w__15m_c'] * budget_df['du_dz_15m_c'] - budget_df['v_w__15m_c'] * budget_df['dv_dz_15m_c']).rename('S')).assign(height=15),
    pd.DataFrame((- budget_df['u_w__20m_c'] * budget_df['du_dz_20m_c'] - budget_df['v_w__20m_c'] * budget_df['dv_dz_20m_c']).rename('S')).assign(height=20)
])

tkeflux_terms_df = pd.concat([
    pd.DataFrame(budget_df['w_tke__2m_c'].rename('tke_flux')).assign(height=2),
    pd.DataFrame(budget_df['w_tke__3m_c'].rename('tke_flux')).assign(height=3),
    pd.DataFrame(budget_df['w_tke__5m_c'].rename('tke_flux')).assign(height=5),
    pd.DataFrame(budget_df['w_tke__10m_c'].rename('tke_flux')).assign(height=10),
    pd.DataFrame(budget_df['w_tke__15m_c'].rename('tke_flux')).assign(height=15),
    pd.DataFrame(budget_df['w_tke__20m_c'].rename('tke_flux')).assign(height=20),
])

In [158]:
budget_terms_df = T_terms_df.reset_index().set_index(['time', 'height']).join(
    S_terms_df.reset_index().set_index(['time', 'height'])
).join(
    tkeflux_terms_df.reset_index().set_index(['time', 'height'])
).join(
    B_terms_df.reset_index().set_index(['time', 'height'])
)
budget_terms_df

,,T,S,tke_flux,B
time,height,,,,
2023-06-08 18:05:00,2,-0.112112,0.052223,-0.498309,0.002379
2023-06-08 18:15:00,2,-0.047569,0.027273,-0.284065,0.001793
2023-06-08 18:25:00,2,-0.000136,0.017808,-0.178662,0.002221
2023-06-08 18:35:00,2,-0.042538,0.017360,-0.159734,0.002362
2023-06-08 18:45:00,2,0.021788,0.008917,-0.121322,0.001305
...,...,...,...,...,...
2023-06-11 17:15:00,20,0.000414,-0.000060,-0.007463,0.000022
2023-06-11 17:25:00,20,0.000249,-0.000113,-0.008791,-0.000109
2023-06-11 17:35:00,20,-0.051991,-0.005134,1.220250,-0.000683


# Calculate terms, normalized 

In [159]:
k = 0.41
norm_factor_2m = k*2/budget_df['u*_2m_c']**3
norm_factor_3m = k*3/budget_df['u*_3m_c']**3
norm_factor_5m = k*5/budget_df['u*_5m_c']**3
norm_factor_10m = k*10/budget_df['u*_10m_c']**3
norm_factor_15m = k*15/budget_df['u*_15m_c']**3
norm_factor_20m = k*20/budget_df['u*_20m_c']**3

In [160]:

reference_temp = ((budget_df['tc_10m_c'].values * units('degC')).to(units('K'))).magnitude
g = 9.81
B_terms_normed_df = pd.concat([
    pd.DataFrame((norm_factor_2m*( g / reference_temp ) * budget_df['w_tc__2m_c']).rename('B')).assign(height = 2),
    pd.DataFrame((norm_factor_3m*( g / reference_temp ) * budget_df['w_tc__3m_c']).rename('B')).assign(height = 3),
    pd.DataFrame((norm_factor_5m*( g / reference_temp ) * budget_df['w_tc__5m_c']).rename('B')).assign(height = 5),
    pd.DataFrame((norm_factor_10m*( g / reference_temp ) * budget_df['w_tc__10m_c']).rename('B')).assign(height = 10),
    pd.DataFrame((norm_factor_15m*( g / reference_temp ) * budget_df['w_tc__15m_c']).rename('B')).assign(height = 15),
    pd.DataFrame((norm_factor_20m*( g / reference_temp ) * budget_df['w_tc__20m_c']).rename('B')).assign(height = 20),
])

T_terms_normed_df = pd.concat([
    pd.DataFrame((norm_factor_2m*(- budget_df['dtkeflux_dz_2m_c'])).rename('T')).assign(height = 2),
    pd.DataFrame((norm_factor_3m*(- budget_df['dtkeflux_dz_3m_c'])).rename('T')).assign(height = 3),
    pd.DataFrame((norm_factor_5m*(- budget_df['dtkeflux_dz_5m_c'])).rename('T')).assign(height = 5),
    pd.DataFrame((norm_factor_10m*(- budget_df['dtkeflux_dz_10m_c'])).rename('T')).assign(height = 10),
    pd.DataFrame((norm_factor_15m*(- budget_df['dtkeflux_dz_15m_c'])).rename('T')).assign(height = 15),
    pd.DataFrame((norm_factor_20m*(- budget_df['dtkeflux_dz_20m_c'])).rename('T')).assign(height = 20)
])
S_terms_normed_df = pd.concat([
    pd.DataFrame((norm_factor_2m*(- budget_df['u_w__2m_c'] * budget_df['du_dz_2m_c'] - budget_df['v_w__2m_c'] * budget_df['dv_dz_2m_c'])).rename('S')).assign(height=2),
    pd.DataFrame((norm_factor_3m*(- budget_df['u_w__3m_c'] * budget_df['du_dz_3m_c'] - budget_df['v_w__3m_c'] * budget_df['dv_dz_3m_c'])).rename('S')).assign(height=3),
    pd.DataFrame((norm_factor_5m*(- budget_df['u_w__5m_c'] * budget_df['du_dz_5m_c'] - budget_df['v_w__5m_c'] * budget_df['dv_dz_5m_c'])).rename('S')).assign(height=5),
    pd.DataFrame((norm_factor_10m*(- budget_df['u_w__10m_c'] * budget_df['du_dz_10m_c'] - budget_df['v_w__10m_c'] * budget_df['dv_dz_10m_c'])).rename('S')).assign(height=10),
    pd.DataFrame((norm_factor_15m*(- budget_df['u_w__15m_c'] * budget_df['du_dz_15m_c'] - budget_df['v_w__15m_c'] * budget_df['dv_dz_15m_c'])).rename('S')).assign(height=15),
    pd.DataFrame((norm_factor_20m*(- budget_df['u_w__20m_c'] * budget_df['du_dz_20m_c'] - budget_df['v_w__20m_c'] * budget_df['dv_dz_20m_c'])).rename('S')).assign(height=20)
])

In [161]:
budget_terms_normed_df = T_terms_normed_df.reset_index().set_index(['time', 'height']).join(
    S_terms_normed_df.reset_index().set_index(['time', 'height'])
).join(
    B_terms_normed_df.reset_index().set_index(['time', 'height'])
)
budget_terms_normed_df

,,T,S,B
time,height,,,
2023-06-08 18:05:00,2,-1.979219,0.921946,0.045879
2023-06-08 18:15:00,2,-1.444208,0.828031,0.059450
2023-06-08 18:25:00,2,-0.006671,0.871620,0.118746
2023-06-08 18:35:00,2,-2.386520,0.973938,0.144738
2023-06-08 18:45:00,2,1.798212,0.735975,0.117682
...,...,...,...,...
2023-06-11 17:15:00,20,0.500979,-0.072297,0.029215
2023-06-11 17:25:00,20,0.460087,-0.209233,-0.221115
2023-06-11 17:35:00,20,-3.368753,-0.332666,-0.048378


# Plot Normed

In [169]:
alt.Chart(
    budget_terms_df.reset_index()
).transform_filter(
    alt.FieldOneOfPredicate('height', [3,5,20])
).transform_fold(
    ['T', 'B', 'S']
).mark_line().encode(
    alt.X('hours(time):T'),
    alt.Y('mean(value):Q'),
    alt.StrokeDash('height:O'),
    alt.Color('key:N'),
)

alt.Chart(...)

In [56]:
April 9

alt.Chart(...)

In [30]:
April 18

alt.Chart(...)

In [ ]:
src = budget_terms_normed_df.reset_index()
src = src[src.time.dt.hour.isin([12,13,14,15,16,17, 18, 19, 20, 21])]
(alt.Chart(src).transform_fold([
    'T', 'S', 'B'
]).transform_filter(
    alt.datum.height > 2
).mark_line(point=True).encode(
    alt.X('mean(value):Q').scale(domain=[-10, 10], clamp=True),
    alt.Y('height:Q'),
    alt.Facet('key:N'),
    alt.Color('hours(time):O'),
    tooltip=('hours(time):O'),
    order='height'
).properties(width=200) |\
alt.Chart(src).transform_filter(
    alt.datum.height > 2
).mark_line(color='black', point=True).encode(
    alt.X('mean(tke_flux):Q'),
    alt.Y('height:Q'),
    alt.Color('hours(time):O'),
    tooltip=('hours(time):O'),
    order='height'
).properties(width=200))

# Plot

In [ ]:
src = budget_terms_df.reset_index()
src = src[src.time.dt.hour.isin([12,13,14,15,16,17, 18, 19, 20, 21])]
(alt.Chart(src).transform_fold([
    'T', 'S', 'B'
]).transform_filter(
    alt.datum.height > 2
).mark_line(point=True).encode(
    alt.X('mean(value):Q'),
    alt.Y('height:Q'),
    alt.Facet('key:N'),
    alt.Color('hours(time):O'),
    tooltip=('hours(time):O'),
    order='height'
).properties(width=200) |\
alt.Chart(src).transform_filter(
    alt.datum.height > 2
).mark_line(color='black', point=True).encode(
    alt.X('mean(tke_flux):Q'),
    alt.Y('height:Q'),
    alt.Color('hours(time):O'),
    tooltip=('hours(time):O'),
    order='height'
).properties(width=200))

In [ ]:
April 9 data

In [ ]:
April 18 data